In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm_notebook

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

from sklearn.model_selection import cross_validate, cross_val_predict, KFold

from matplotlib import rc
import matplotlib.pyplot as plt
%matplotlib inline
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings 
warnings.filterwarnings("ignore")

### Trial no.1 

string -> 0
missing -> 0

minmaxscaler

lightGBM

```
param = {
        'objective': 'multiclass',
        'num_class': 198,
        'boosting':'goss', 
         # Gradient-based One-Side Sampling 
         # 일반적인 의사결정나무 부스팅(gbdt)보다 빠른 수렴이 가능합니다.
         # 하지만 과최적화에 빠지기 더 쉽습니다.  
        'num_leaves':31, 
#         'max_depth':5,
        'min_data_in_leaf':20,
        'metric':'multi_logloss', 
        }

evals_result0={} 
num_round = 100 

lgbst = lgb.train(param, 
                  train_data, 
                  num_round, 
                  valid_sets=[valid_data], 
                  evals_result=evals_result0, 
                  early_stopping_rounds=20, 
                  verbose_eval=5)
```

In [18]:
train = pd.read_csv('../../../data/train/train_100-dacon_baseline_preprocess.csv')

In [19]:
train = train[train['label'] != 999].reset_index(drop=True)

In [23]:
train

,id,time,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,...,V5112,V5113,V5114,V5115,V5116,V5117,V5118,V5119,V5120,label
0,545,10,30.472197,8.695875,8.703739,8.709477,8.711114,200.381358,159.282182,-5.018619e-19,...,1.0,1.0,1.0,60.0,0.0,0.0,-0.000002,85.4,0.0,20
1,545,11,30.451815,8.635427,8.705436,8.699428,8.711520,181.601175,156.666622,6.360338e-19,...,1.0,1.0,1.0,60.0,0.0,0.0,0.000003,85.4,0.0,20
2,545,12,30.464503,8.743024,8.725929,8.671431,8.743335,198.761704,191.961581,1.067647e-20,...,1.0,1.0,1.0,60.0,0.0,0.0,-0.000028,85.4,0.0,20
3,545,13,30.483019,8.752263,8.711617,8.776546,8.703115,169.724555,177.933862,-5.364949e-19,...,1.0,1.0,1.0,60.0,0.0,0.0,-0.000009,85.4,0.0,20
4,545,14,30.473983,8.740589,8.692430,8.740275,8.717582,186.863251,170.349628,-1.922528e-19,...,1.0,1.0,1.0,60.0,0.0,0.0,0.000010,85.4,0.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74515,548,95,30.484728,8.671523,8.693305,8.739597,8.713562,199.417463,233.000286,-8.062924e-19,...,1.0,1.0,1.0,60.0,0.0,0.0,0.000017,85.4,0.0,21
74516,548,96,30.472393,8.814420,8.687622,8.709898,8.728637,232.612801,197.675025,-2.410963e-19,...,1.0,1.0,1.0,60.0,0.0,0.0,-0.000023,85.4,0.0,21
74517,548,97,30.472769,8.725393,8.678891,8.678229,8.689200,215.332668,237.532353,-9.438657e-20,...,1.0,1.0,1.0,60.0,0.0,0.0,0.000006,85.4,0.0,21
74518,548,98,30.484135,8.719626,8.702743,8.674975,8.682801,214.755569,190.775921,-2.170485e-19,...,1.0,1.0,1.0,60.0,0.0,0.0,-0.000008,85.4,0.0,21


In [25]:
train_label = train.label
train_id = train.id
train_time = train.time

In [26]:
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
X_train = minmax.fit_transform(train.drop(['id','time','label'], axis=1))

In [27]:
X_train

array([[9.64050500e-01, 4.12195445e-01, 4.14657678e-01, ...,
        7.65708716e-07, 1.00000000e+00, 0.00000000e+00],
       [9.63405681e-01, 4.06857876e-01, 4.14813524e-01, ...,
        8.25627024e-07, 1.00000000e+00, 0.00000000e+00],
       [9.63807086e-01, 4.16358619e-01, 4.16695952e-01, ...,
        3.88276499e-07, 1.00000000e+00, 0.00000000e+00],
       ...,
       [9.64068594e-01, 4.14801851e-01, 4.12375101e-01, ...,
        8.69692807e-07, 1.00000000e+00, 0.00000000e+00],
       [9.64428187e-01, 4.14292559e-01, 4.14566152e-01, ...,
        6.82011930e-07, 1.00000000e+00, 0.00000000e+00],
       [9.63603275e-01, 4.15180376e-01, 4.15061330e-01, ...,
        7.08726075e-07, 1.00000000e+00, 0.00000000e+00]])

In [28]:
train_label

0        20
1        20
2        20
3        20
4        20
         ..
74515    21
74516    21
74517    21
74518    21
74519    21
Name: label, Length: 74520, dtype: int64

In [29]:
X_train, X_valid = train_test_split(X_train, test_size = .25, random_state=42)
y_train, y_valid = train_test_split(train_label, test_size = .25, random_state=42)

In [38]:
len(set(train_label))

198

In [36]:
len(set(list(y_train)))

198

In [41]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train) #, feature_name=X_train.columns)
valid_data = lgb.Dataset(X_valid, label=y_valid) #, feature_name=X_valid.columns)

param = {
        'objective': 'multiclass',
        'num_class': 198,
        'boosting':'goss', 
         # Gradient-based One-Side Sampling 
         # 일반적인 의사결정나무 부스팅(gbdt)보다 빠른 수렴이 가능합니다.
         # 하지만 과최적화에 빠지기 더 쉽습니다.  
        'num_leaves':31, 
#         'max_depth':5,
        'min_data_in_leaf':20,
        'metric':'multi_logloss', 
        }

evals_result0={} 
num_round = 100 

lgbst = lgb.train(param, 
                  train_data, 
                  num_round, 
                  valid_sets=[valid_data], 
                  evals_result=evals_result0, 
                  early_stopping_rounds=20, 
                  verbose_eval=5)

Training until validation scores don't improve for 20 rounds.
[5]	valid_0's multi_logloss: 5.29644
[10]	valid_0's multi_logloss: 6.40586
[15]	valid_0's multi_logloss: 17.3349
[20]	valid_0's multi_logloss: 23.5062
Early stopping, best iteration is:
[1]	valid_0's multi_logloss: 1.40871


# 추론 ??????

In [86]:
import os
import pandas as pd 
import numpy as np
import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool 
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할

import sys
sys.path.append('../utils/')
from data_loader_v2 import data_loader_v2 # 자체적으로 만든 data loader version 2.0 ([데이콘 15회 대회] 데이터 설명 및 데이터 불러오기 영상 참조)

In [87]:
test_folder = '../../../data/test/'
test_list = os.listdir(test_folder)

In [88]:
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    return combined_df

test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=10, nrows=60)

In [90]:
for col in tqdm_notebook(test.columns):
    if test[col].dtype != 'float64':
        test[col] = pd.to_numeric(test[col], errors='coerce')

In [91]:
test = test.replace(np.nan, 0, regex=True)
test = test.fillna(value=0)

In [100]:
test

,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,V0009,...,V5111,V5112,V5113,V5114,V5115,V5116,V5117,V5118,V5119,V5120
id,,,,,,,,,,,,,,,,,,,,,
1012,30.478099,8.750725,8.697041,8.743622,8.721824,187.977806,179.552075,1.637091e-20,0.0,0.000265,...,1.0,1.0,1.0,1.0,60.0,0.0,0.0,6.775936e-06,85.4,0.0
1012,30.484229,8.697382,8.706707,8.709945,8.692371,180.573766,232.979942,1.969044e-20,0.0,0.001324,...,1.0,1.0,1.0,1.0,60.0,0.0,0.0,6.025410e-06,85.4,0.0
1012,30.472894,8.803095,8.701016,8.699278,8.708927,200.110774,185.582184,-2.967308e-19,0.0,0.000735,...,1.0,1.0,1.0,1.0,60.0,0.0,0.0,6.435280e-07,85.4,0.0
1012,30.482065,8.715436,8.718378,8.713104,8.695852,212.349745,179.912506,-1.136840e-18,0.0,0.000666,...,1.0,1.0,1.0,1.0,60.0,0.0,0.0,8.701290e-06,85.4,0.0
1012,30.474575,8.721665,8.725964,8.745781,8.706930,168.468533,165.053369,-5.193783e-19,0.0,0.000532,...,1.0,1.0,1.0,1.0,60.0,0.0,0.0,7.862742e-07,85.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,30.453184,8.825302,8.679480,8.728759,8.715055,214.782495,217.011876,-5.498920e-20,0.0,-0.002533,...,1.0,1.0,1.0,1.0,60.0,0.0,0.0,-7.258669e-06,85.4,0.0
1037,30.470358,8.623486,8.726907,8.716872,8.696205,203.697865,144.390101,4.580207e-19,0.0,0.000271,...,1.0,1.0,1.0,1.0,60.0,0.0,0.0,1.660617e-05,85.4,0.0
1037,30.478986,8.675820,8.722456,8.710132,8.676874,199.043710,194.637121,8.475931e-19,0.0,0.000838,...,1.0,1.0,1.0,1.0,60.0,0.0,0.0,6.474862e-06,85.4,0.0


In [94]:
X_test = minmax.fit_transform(test)

In [95]:
pred = lgbst.predict(X_test,  num_iteration=lgbst.best_iteration)

In [104]:
submission = pd.DataFrame(data=pred)
submission.index = test.index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()
submission = submission.reset_index()
submission

In [107]:
submission.to_csv('submission_wy_trial_1.csv', index=False)

In [ ]:
for i in range(828,1548):
    print(i)
    if i != 1154 and i != 1168:
        tmp = pd.read_csv('./test/'+str(i)+'.csv')
        tmp = tmp.drop(['time'], axis=1)
        tmp = minmax.transform(tmp) #self normalize 할시 minmax_self.fit_transform으로 변경
        tmp = pca.transform(tmp) # pca pc 개수 변경시 컬럼수 조정 필요.
        result = rfc.predict_proba(tmp)

        results.append(np.mean(result,axis=0))

In [ ]:
lgbst.predict(X_test, num_iteration=lgbst.best_iteration)

In [ ]:
y_pred_lgb = lgbst.predict(X_test,  num_iteration=lgbst.best_iteration)
target_names=['0', '1']

y_pred_2 = list(map(lambda x: 1 if x >0.5 else 0,  y_pred_lgb))

# print("\n",confusion_matrix(y_test, a)) 
print(classification_report(y_test['TimeBeforeDefault2'], y_pred_2, target_names=target_names))

In [ ]:
fpr_lgb, tpr_lgb, threshold_lgb =  roc_curve(y_test['TimeBeforeDefault2'], y_pred_lgb)
auc_lgb = auc(fpr_lgb, tpr_lgb)
print("Test AUC :", auc_lgb)

In [ ]:
def get_lgb_feature_importances(lgbst):
    feature_imp = pd.DataFrame(sorted(zip(lgb.feature_importance(lgbst),X.columns)), columns=['Value','Feature'])
    plt.figure(figsize=(20, 10))
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.show()

In [ ]:
lgb.plot_importance(lgbst, max_num_features=20)